In [1]:
import uuid
# project_id = str(uuid.uuid4())
run_id = str(uuid.uuid4())
print(run_id)

16e3f179-bb70-4632-9a39-9a6fac4a5581


In [ ]:
# payload-1 (classification Airline Delay dataset)
exp_data = '''{{
"source":"EXP", 
"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", 
"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", 
"run_id":"{0}", 
"exp_name": "Final_recipe", 
"algorithm_type":"classification", 
"algo_details": {{"snowflake.ml.modeling.xgboost.XGBClassifier": null}}, 
"dataset": "AIRLINE_DEP_DELAY_10K", 
"target_column": "DEP_DEL15"}}'''.format(run_id)

In [3]:
# payload-2 (Regression Alcohol dataset)
exp_data = '''{{
"source":"EXP", 
"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", 
"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", 
"run_id":"{0}", 
"exp_name": "Final_recipe", 
"algorithm_type":"regression", 
"algo_details": {{"snowflake.ml.modeling.ensemble.RandomForestRegressor": null}}, 
"dataset": "ALCOHOL_DATA_10L", 
"target_column": "QUALITY"}}'''.format(run_id)

In [ ]:
# payload-3 (classification Employee dataset)
exp_data = '''{{
"source":"EXP", 
"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", 
"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", 
"run_id":"{0}", 
"exp_name": "Final_recipe", 
"algorithm_type":"classification", 
"algo_details": {{"snowflake.ml.modeling.linear_model.LogisticRegression": null}}, 
"dataset": "EMPLOYEE_10L", 
"target_column": "LEAVEORNOT"}}'''.format(run_id)

In [ ]:
# payload-4 (Regression Diamonds dataset)
exp_data = '''{{
"source":"EXP", 
"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", 
"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", 
"run_id":"{0}", 
"exp_name": "Final_recipe", 
"algorithm_type":"regression", 
"algo_details": {{"snowflake.ml.modeling.xgboost.XGBRegressor": null}}, 
"dataset": "DIAMONDS", 
"target_column": "PRICE"}}'''.format(run_id)

In [4]:
exp_data

'{\n"source":"EXP", \n"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", \n"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", \n"run_id":"16e3f179-bb70-4632-9a39-9a6fac4a5581", \n"exp_name": "Final_recipe", \n"algorithm_type":"regression", \n"algo_details": {"snowflake.ml.modeling.ensemble.RandomForestRegressor": null}, \n"dataset": "ALCOHOL_DATA_10L", \n"target_column": "QUALITY"}'

In [9]:
from snowflake.snowpark.session import Session
import snow_exp as sf
import json, asyncio

In [ ]:
#API function
async def execute_sf_sproc(payload, sproc_name="run_experiment"):
    exp_details = json.loads(payload)
    session = sf.get_session(exp_details.get("dataset"),exp_details.get("project_id"))
    stage = sf.create_stage(session)
    sf.create_sproc(session, stage)
    try:
        
    return "Success"

async def initiate_sproc_process(sproc_name):
            sproc_response = session.call(sproc_name, payload)
            return sproc_response

In [1]:
%%time
# Initilization
# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print("Creating Snowflake Session object...")
exp_details=json.loads(exp_data)
session = get_session(exp_details.get("dataset"),exp_details.get("project_id"))
stage = create_stage(session)
print("Session has been created !")

print("Creating stored procedure...")
session.custom_package_usage_config['enabled'] = True
session.sproc.register(func=run_experiment,
                       name="run_experiment",
                       packages=["snowflake-snowpark-python", "snowflake-ml-python","scikit-learn"],
                       isPermanant=False,
                       stage_location=stage,
                       replace=True)
print("Stored procedure has been created successfully!")

# tagging session
print("Setting tag to session object: tag= ", run_id)
session.query_tag=run_id

print("Executing Procedure")
procedure_response = session.call("run_experiment", exp_data)
# procedure_response = run_experiment(session, exp_data)
print("Stored Procedure Executed Successfully !")
print(procedure_response)

#Log in mlflow
print("Logging in mlflow completed !")

Creating Snowflake Session object...


NameError: name 'json' is not defined